In [1]:
import requests
import pandas as pd
import json

In [2]:
URL="https://www.dlsu.edu.ph"

In [3]:
page = requests.get(URL)

# Extract all sub-urls from dlsu.edu.ph

In [4]:
import requests
import html5lib
from bs4 import BeautifulSoup
import re
import time

soup = BeautifulSoup(page.content, 'lxml')
data = []
links = []

url_limit = 50

start = time.time()

def remove_duplicates(l): # remove duplicates and unURL string
    for item in l:
        match = re.search("(?P<url>https?://[^\s]+)", item)
        if match is not None:
            links.append((match.group("url")))


for link in soup.find_all('a', href=True):
    data.append(str(link.get('href')))
    
flag = True
remove_duplicates(data)

while flag:
    try:
        for link in links:
            for j in soup.find_all('a', href=True):
                temp = []
                source_code = requests.get(link)
                soup = BeautifulSoup(source_code.content, 'lxml')
                temp.append(str(j.get('href')))
                remove_duplicates(temp)

                if len(links) > url_limit: # set limitation to number of URLs
                    break
            if len(links) > url_limit:
                break
        if len(links) > url_limit:
            break
    except Exception as e:
        print(e)
        if len(links) > url_limit:
            break
            
for url in links:
    print(url)
    
print ("Elapsed Time: %s" % (time.time() - start))

https://www.dlsu.edu.ph/about-dlsu/
https://www.dlsu.edu.ph/about-dlsu/ourcorevalues/
https://www.dlsu.edu.ph/offices/qao/accreditation/
https://www.dlsu.edu.ph/offices/office-of-the-chancellor/
https://www.dlsu.edu.ph/university-fellows/
https://www.dlsu.edu.ph/emeriti/
https://www.dlsu.edu.ph/brothers/
https://www.dlsu.edu.ph/lasallianmission/
https://www.dlsu.edu.ph/offices/
https://www.dlsu.edu.ph/inside/board-advisors/
https://www.dlsu.edu.ph/admissions/scholarships/
https://www.dlsu.edu.ph/admissions/graduate/
https://www.dlsu.edu.ph/integrated-school/
https://www.dlsu.edu.ph/integrated-school/
https://www.dlsu.edu.ph/integrated-school/
https://www.dlsu.edu.ph/integratedschool-test/senior-high-school/
https://www.dlsu.edu.ph/colleges/
https://www.dlsu.edu.ph/colleges/gcoe/overview/
http://www.dlsu.edu.ph/wp-content/uploads/pdf/laguna-campus/dlsu-laguna-campus-viewbook.pdf
https://www.dlsu.edu.ph/admissions/graduate/graduate-viewbooks/
https://www.dlsu.edu.ph/wp-content/uploads/pd

In [5]:
web_index = 10 # for testing purposes
print(links[web_index])

https://www.dlsu.edu.ph/admissions/scholarships/


# Get Name & Email

In [6]:
def deCFEmail(fp):
    try:
        r = int(fp[:2],16)
        email = ''.join([chr(int(fp[i:i+2], 16) ^ r) for i in range(2, len(fp), 2)])
        return email
    except (ValueError):
        pass

In [71]:
emailList = []
nameList = []

def emailNameExtractor(urlString):
    getH=requests.get(urlString)
    h=getH.content
    soup=BeautifulSoup(h,'html.parser')
    mailtos = soup.select('a[href]')
    for i in mailtos:
        href=i['href']
        if "email-protect" in href:
            emailList.append(deCFEmail(href.split("#")[1]))
            nameList.append(i.text)

In [82]:
soup

<!DOCTYPE html>

<html itemscope="" itemtype="http://schema.org/WebPage" lang="en-US">
<head><link href="https://www.dlsu.edu.ph/wp-content/cache/minify/91abb.default.include.18995b.css" media="all" rel="stylesheet"/>
<script defer="" src="https://www.dlsu.edu.ph/wp-content/cache/minify/91abb.default.include.51581c.js"></script>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="http://gmpg.org/xfn/11" rel="profile"/>
<link href="https://www.dlsu.edu.ph/xmlrpc.php" rel="pingback"/>
<title>Office of the Dean - De La Salle University</title>
<!-- This site is optimized with the Yoast SEO plugin v11.6 - https://yoast.com/wordpress/plugins/seo/ -->
<meta content="DLSU provides world-class leaders that shape the College of Computer Studies. View and learn more about them here." name="description">
<link href="https://www.dlsu.edu.ph/colleges/ccs/office-of-the-dean/" rel="canonical">
<meta content="en_US" property="og:locale">
<meta cont

In [79]:
emailNameExtractor('https://www.dlsu.edu.ph/colleges/ccs/office-of-the-dean/')
emailList

['rafael.cabredo@dlsu.edu.ph',
 'charibeth.cheng@dlsu.edu.ph',
 'shirley.chu@dlsu.edu.ph',
 'christine.diane.ramos@dlsu.edu.ph',
 'ma.rowena.caguiat@dlsu.edu.ph',
 'gregory.cu@dlsu.edu.ph',
 'marnel.peradilla@dlsu.edu.ph',
 'arlyn.ong@dlsu.edu.ph',
 'ronald.pascual@dlsu.edu.ph',
 'arlyn.ong@dlsu.edu.ph',
 'clement.ong@dlsu.edu.ph',
 'michelle.ching@dlsu.edu.ph',
 'lissa.magpantay@dlsu.edu.ph',
 'oliver.malabanan@dlsu.edu.ph',
 'michelle.ching@dlsu.edu.ph',
 'lissa.magpantay@dlsu.edu.ph',
 'briane.samson@dlsu.edu.ph',
 'neil.delgallego@dlsu.edu.ph',
 'edward.tighe@dlsu.edu.ph',
 'ethel.ong@dlsu.edu.ph',
 'neil.delgallego@dlsu.edu.ph',
 'ma.christine.gendrano@dlsu.edu.ph',
 'ryan.dimaunahan@dlsu.edu.ph',
 'rafael.cabredo@dlsu.edu.ph',
 'charibeth.cheng@dlsu.edu.ph',
 'shirley.chu@dlsu.edu.ph',
 'christine.diane.ramos@dlsu.edu.ph',
 'ma.rowena.caguiat@dlsu.edu.ph',
 'gregory.cu@dlsu.edu.ph',
 'marnel.peradilla@dlsu.edu.ph',
 'arlyn.ong@dlsu.edu.ph',
 'ronald.pascual@dlsu.edu.ph',
 'arlyn.

In [80]:
nameList

['Dr. Rafael A. Cabredo',
 'Dr. Charibeth K. Cheng',
 'Ms. Shirley B. Chu',
 'Dr. Christine Diane L. Ramos',
 'Dr. Ma. Rowena R. Caguiat',
 'Mr. Gregory G. Cu',
 'Dr. Marnel S. Peradilla',
 'Ms. Arlyn Verina L. Ong-Tiu',
 'Dr. Ronald M. Pascual',
 'Ms. Arlyn Verina L. Ong-Tiu',
 'Mr. Clement Y. Ong',
 'Dr. Michelle Renee D. Ching',
 'Ms. Lissa Andrea K. Magpantay',
 'Mr. Oliver A. Malabanan',
 'Dr. Michelle Renee D. Ching',
 'Ms. Lissa Andrea K. Magpantay',
 'Dr. Briane Paul V. Samson',
 'Mr. Neil Patrick A. Del Gallego',
 'Mr. Edward P Tighe',
 'Dr. Ethel C. Ong',
 'Mr. Neil Patrick A. Del Gallego',
 'Ms. Ma. Christine A. Gendrano',
 'Mr. Ryan Samuel M. Dimaunahan',
 'Dr. Rafael A. Cabredo',
 'Dr. Charibeth K. Cheng',
 'Ms. Shirley B. Chu',
 'Dr. Christine Diane L. Ramos',
 'Dr. Ma. Rowena R. Caguiat',
 'Mr. Gregory G. Cu',
 'Dr. Marnel S. Peradilla',
 'Ms. Arlyn Verina L. Ong-Tiu',
 'Dr. Ronald M. Pascual',
 'Ms. Arlyn Verina L. Ong-Tiu',
 'Mr. Clement Y. Ong',
 'Dr. Michelle Renee D

# Combined Email and Name List

In [76]:
finalList = list(zip(nameList,emailList))
finalList = set(finalList)
finalList

{('Dr. Briane Paul V. Samson', 'briane.samson@dlsu.edu.ph'),
 ('Dr. Charibeth K. Cheng', 'charibeth.cheng@dlsu.edu.ph'),
 ('Dr. Christine Diane L. Ramos', 'christine.diane.ramos@dlsu.edu.ph'),
 ('Dr. Ethel C. Ong', 'ethel.ong@dlsu.edu.ph'),
 ('Dr. Ma. Rowena R. Caguiat', 'ma.rowena.caguiat@dlsu.edu.ph'),
 ('Dr. Marnel S. Peradilla', 'marnel.peradilla@dlsu.edu.ph'),
 ('Dr. Michelle Renee D. Ching', 'michelle.ching@dlsu.edu.ph'),
 ('Dr. Rafael A. Cabredo', 'rafael.cabredo@dlsu.edu.ph'),
 ('Dr. Ronald M. Pascual', 'ronald.pascual@dlsu.edu.ph'),
 ('Mr. Clement Y. Ong', 'clement.ong@dlsu.edu.ph'),
 ('Mr. Edward P Tighe', 'edward.tighe@dlsu.edu.ph'),
 ('Mr. Gregory G. Cu', 'gregory.cu@dlsu.edu.ph'),
 ('Mr. Neil Patrick A. Del Gallego', 'neil.delgallego@dlsu.edu.ph'),
 ('Mr. Oliver A. Malabanan', 'oliver.malabanan@dlsu.edu.ph'),
 ('Mr. Ryan Samuel M. Dimaunahan', 'ryan.dimaunahan@dlsu.edu.ph'),
 ('Ms. Arlyn Verina L. Ong-Tiu', 'arlyn.ong@dlsu.edu.ph'),
 ('Ms. Lissa Andrea K. Magpantay', 'liss

# Threading 

## wala pang ginagawa to

In [9]:
import threading
import time

start = time.time()

def fetch_url(url):
    print ("'%s\' fetched in %ss" % (url, (time.time() - start)))
    
threads = [threading.Thread(target=fetch_url, args=(url,)) for url in links]

for thread in threads:
    thread.start()
    
for thread in threads:
    thread.join()

print ("Elapsed Time: %s" % (time.time() - start))

'https://www.dlsu.edu.ph/about-dlsu/' fetched in 0.0009992122650146484s
'https://www.dlsu.edu.ph/about-dlsu/ourcorevalues/' fetched in 0.0019991397857666016s
'https://www.dlsu.edu.ph/offices/qao/accreditation/' fetched in 0.014489412307739258s
'https://www.dlsu.edu.ph/offices/office-of-the-chancellor/' fetched in 0.015494346618652344s
'https://www.dlsu.edu.ph/university-fellows/' fetched in 0.01648998260498047s
'https://www.dlsu.edu.ph/emeriti/' fetched in 0.017490386962890625s
'https://www.dlsu.edu.ph/brothers/' fetched in 0.018489360809326172s
'https://www.dlsu.edu.ph/lasallianmission/' fetched in 0.019489765167236328s
'https://www.dlsu.edu.ph/offices/' fetched in 0.02048945426940918s
'https://www.dlsu.edu.ph/inside/board-advisors/' fetched in 0.021489381790161133s
'https://www.dlsu.edu.ph/admissions/scholarships/' fetched in 0.022489309310913086s
'https://www.dlsu.edu.ph/admissions/graduate/' fetched in 0.023488759994506836s
'https://www.dlsu.edu.ph/integrated-school/' fetched in 0.

# Get emails protected by cloudflare

# Pwede na ata di isama to kasi ayaw gumana

In [10]:
mydivs = soup.findAll("span",{"class": "__cf_email__"})

length = len(mydivs)

for x in range(length):
    a = cfDecodeEmail(mydivs[x]["data-cfemail"])
    print(a)


NameError: name 'cfDecodeEmail' is not defined

In [ ]:
print (mydivs[1]["data-cfemail"])
print(len(mydivs))